### 01PRE.ipynb
repeatedly run this notebook by changing the varible "filename" 
- filename = 'dataset/train.tsv'
- filename = 'dataset/valid.tsv'
- filename = 'dataset/testA.tsv'
- filename = 'dataset/testB.tsv'

05/27/2020

In [1]:
filename = 'dataset/testB.tsv'
import pandas as pd
df = pd.read_csv(filename, quoting=3, sep='\t')

In [2]:
# remove queries like "'s xxx" as reported in the forum
print(len(df))
df = df[~df['query'].str.startswith('\'s')]
print(len(df))

29005
29005


In [3]:
import numpy as np
import base64
def decode(row):
    row['features'] = np.frombuffer(base64.b64decode(row['features']), 
                                 dtype=np.float32).reshape(row['num_boxes'], 2048).astype(np.float16)
    row['boxes'] = np.frombuffer(base64.b64decode(row['boxes']), 
                                 dtype=np.float32).reshape(row['num_boxes'], 4).astype(np.float16)
    row['class_labels'] = np.frombuffer(base64.b64decode(row['class_labels']), 
                                 dtype=np.int64).reshape(row['num_boxes'])
    return row
df = df.apply(decode, axis=1)

In [4]:
from tokenizers import BertWordPieceTokenizer
tokenizer = BertWordPieceTokenizer("dataset/bert-base-cased-vocab.txt", lowercase=False)
print(tokenizer)
def query2tokenid(query):
    return tokenizer.encode(query).ids
df['query_tokenid'] = df['query'].apply(query2tokenid)

Tokenizer(vocabulary_size=28996, model=BertWordPiece, add_special_tokens=True, unk_token=[UNK], sep_token=[SEP], cls_token=[CLS], clean_text=True, handle_chinese_chars=True, strip_accents=True, lowercase=False, wordpieces_prefix=##)


In [5]:
df.to_pickle(filename.replace('tsv', 'pkl').replace('dataset', 'extracted'))

In [18]:
# using 10% of train for faster development
# df.sample(300000).to_pickle('extracted/train_tiny.pkl')

In [11]:
df

,product_id,image_h,image_w,num_boxes,boxes,features,class_labels,query,query_id,query_tokenid
0,103042011,800,800,9,"[[111.0, 428.0, 274.0, 541.0], [174.0, 125.0, ...","[[0.0, 0.02663, 0.2598, 0.0, 0.2322, 0.0, 0.0,...","[28, 28, 28, 28, 28, 9, 28, 28, 28]",wooden file cabinet,0,"[101, 4122, 4956, 6109, 102]"
1,101177314,750,750,4,"[[245.0, 330.0, 335.0, 449.0], [5.0, 198.0, 73...","[[0.0, 0.0, 6.926, 0.0, 0.437, 0.0, 0.0, 0.0, ...","[28, 9, 28, 28]",wooden file cabinet,0,"[101, 4122, 4956, 6109, 102]"
2,103009949,800,800,6,"[[138.0, 522.0, 709.0, 782.0], [405.0, 428.0, ...","[[0.0, 0.0, 0.0138, 0.0, 0.011536, 0.0, 0.0, 0...","[28, 28, 28, 28, 28, 28]",wooden file cabinet,0,"[101, 4122, 4956, 6109, 102]"
3,103022608,800,800,7,"[[423.0, 460.0, 506.0, 533.0], [123.0, 149.0, ...","[[0.0, 0.1203, 0.9106, 0.0, 0.6167, 0.0, 0.0, ...","[28, 28, 28, 28, 9, 9, 9]",wooden file cabinet,0,"[101, 4122, 4956, 6109, 102]"
4,103041034,800,800,5,"[[133.0, 144.0, 347.0, 534.0], [134.0, 92.0, 6...","[[0.0, 0.05518, 0.000805, 0.0, 0.06665, 0.0, 0...","[28, 9, 28, 28, 28]",wooden file cabinet,0,"[101, 4122, 4956, 6109, 102]"
...,...,...,...,...,...,...,...,...,...,...
29000,103032444,1000,1000,10,"[[179.0, 52.0, 680.0, 432.0], [183.0, 559.0, 6...","[[0.0, 0.0, 1.07, 0.0, 0.003616, 0.0, 0.0, 0.0...","[0, 0, 29, 29, 2, 2, 4, 4, 4, 4]",hedging loose sweater,999,"[101, 1119, 13556, 5768, 15195, 102]"
29001,103018483,1000,1000,2,"[[192.0, 246.0, 365.0, 350.0], [295.0, 168.0, ...","[[0.0, 0.0, 1.14, 0.0, 0.0, 0.287, 0.0, 0.0, 0...","[29, 0]",hedging loose sweater,999,"[101, 1119, 13556, 5768, 15195, 102]"
29002,103011147,800,800,1,"[[96.0, 48.0, 694.0, 732.0]]","[[0.0, 0.0, 0.3809, 0.0, 0.0, 0.0, 0.0, 0.0, 0...",[0],hedging loose sweater,999,"[101, 1119, 13556, 5768, 15195, 102]"
29003,103031908,800,800,1,"[[0.0, 0.0, 664.0, 800.0]]","[[0.0, 0.0, 0.01732, 0.0, 0.0, 0.5107, 0.0, 0....",[0],hedging loose sweater,999,"[101, 1119, 13556, 5768, 15195, 102]"


In [6]:
## finally we have
!ls -lh extracted

total 50G
-rw-rw-r-- 1 amax amax 458M 5月  27 22:43 testA.pkl
-rw-rw-r-- 1 amax amax 465M 6月  10 22:43 testB.pkl
-rw-rw-r-- 1 amax amax  44G 5月  27 22:41 train.pkl
-rw-rw-r-- 1 amax amax 4.4G 5月  27 22:42 train_tiny.pkl
-rw-rw-r-- 1 amax amax 234M 5月  27 22:42 valid.pkl
